In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/odb-final-task/sample_submission.csv
/kaggle/input/odb-final-task/model.csv
/kaggle/input/odb-final-task/data.csv


In [2]:
# Standard python libraries
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns



# Data loading

In [3]:
%%time

data = pd.read_csv('/kaggle/input/odb-final-task/data.csv')
model_data = pd.read_csv("/kaggle/input/odb-final-task/model.csv")
submission = pd.read_csv("/kaggle/input/odb-final-task/sample_submission.csv")

CPU times: user 19.9 s, sys: 5.46 s, total: 25.3 s
Wall time: 28 s


In [4]:
submission.head()

,0.5488135039273248
0,0.715189
1,0.602763
2,0.544883
3,0.423655
4,0.645894


In [5]:
%%time

data.shape, model_data.shape, submission.shape

CPU times: user 26 µs, sys: 7 µs, total: 33 µs
Wall time: 36 µs


((82617, 1910), (82617, 2), (999, 1))

In [6]:
df = data.merge(model_data, on='incident')

In [7]:
df.head()

,incident,flg_90_12_add,APPLICATION_MONTH,Category_Feature_0,Category_Feature_1,Category_Feature_2,Category_Feature_3,Category_Feature_4,Category_Feature_5,Category_Feature_6,...,Feature_1879,Feature_1880,Feature_1881,Feature_1882,Feature_1883,Feature_1884,Feature_1885,Feature_1886,Feature_1887,PD
0,14511110,0.0,2018-10,6.0,2,25,2.0,0.0,3,1,...,NaN,NaN,NaN,NaN,0,0,0,1,20184,0.022191
1,15018800,0.0,2018-12,1.0,2,49,4.0,0.0,3,1,...,0.259385,0.547065,0.28768,0.28768,0,1,1,1,20184,0.019972
2,14499737,0.0,2018-10,6.0,2,24,1.0,0.0,3,0,...,0.420321,0.242687,NaN,NaN,0,1,0,1,20184,0.043884
3,14953427,0.0,2018-12,4.0,2,26,1.0,0.0,3,1,...,NaN,NaN,NaN,NaN,0,1,1,1,20184,0.013412
4,14744453,0.0,2018-11,6.0,2,30,4.0,2.0,3,1,...,0.045863,NaN,NaN,NaN,0,1,0,1,20184,0.033354


In [8]:
df['target'] = abs(df['PD'] - df['flg_90_12_add'])

In [9]:
data = df[df['target'].notnull()]

In [10]:
data = data.drop(columns = ['flg_90_12_add', 'PD', 'APPLICATION_MONTH'])

In [11]:
test = df[df['target'].isnull()]

In [12]:
test = test.drop(columns = ['flg_90_12_add', 'PD', 'APPLICATION_MONTH'])

In [13]:
data.shape, test.shape

((81617, 1909), (1000, 1909))

In [14]:
import lightgbm as lgb

In [15]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            df[col].fillna(0, inplace=True)         
    return df

train = fill_na(data)
test = fill_na(test)

In [16]:
y = train['target']

In [17]:
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=0)

In [18]:
#from lightgbm import LGBMRegressor

In [19]:
from sklearn.metrics import mean_absolute_error
import math

In [20]:
#lgbm = LGBMRegressor(objective='regression', 
#                                       num_leaves=4,
#                                       learning_rate=0.01, 
#                                       n_estimators=12000, 
#                                       max_bin=200, 
#                                       bagging_fraction=0.75,
#                                       bagging_freq=5, 
#                                       bagging_seed=7,
#                                       feature_fraction=0.4, 
#                                       )
#lgbm.fit(X_train, y_train,eval_metric='rmse')
#predictions = lgbm.predict(X_test)
#print('Root Mean Square Error test = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predictions))))

In [21]:
from xgboost import XGBRegressor

In [22]:
xgb =XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=4, min_child_weight=1.5, n_estimators=2400,
             n_jobs=1, nthread=None, objective='reg:linear',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)
xgb.fit(X_train, y_train)
predictions = xgb.predict(X_test)
print('Root Mean Square Error test = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predictions))))

[20:57:07] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean Square Error test = 0.0030050645488292026


In [23]:
predicted_1 = xgb.predict(test.values)

In [24]:
submission = pd.DataFrame({
        "target": predicted_1
    })
submission.to_csv('submission.csv', index=False)